### 1. 引入大模型

In [1]:
from utils import get_qwen_models

In [2]:
llm, chat, embed = get_qwen_models()

### 2. 引入各种文档加载器
- 传入相关参数，实例化加载器
- 调用 load 方法， 真正去加载内容
- 注意：返回一个包含 Document 对象的列表

In [3]:
from langchain_community.document_loaders import CSVLoader
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import UnstructuredURLLoader
from langchain_community.document_loaders import UnstructuredHTMLLoader
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.document_loaders import UnstructuredExcelLoader
from langchain_community.document_loaders import UnstructuredPowerPointLoader
from langchain_community.document_loaders import UnstructuredWordDocumentLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [4]:
students_loader = CSVLoader(file_path="students.csv")

In [5]:
docs = students_loader.load()

In [6]:
docs

[Document(metadata={'source': 'students.csv', 'row': 0}, page_content='name: Tom\nage: 12\nscore: 77'),
 Document(metadata={'source': 'students.csv', 'row': 1}, page_content='name: Jerry\nage: 11\nscore: 88'),
 Document(metadata={'source': 'students.csv', 'row': 2}, page_content='name: Jim\nage: 12\nscore: 96')]

In [7]:
"""
    RAG系统
"""
from langchain_core.output_parsers import StrOutputParser
# 可执行的占位符
from langchain_core.runnables import RunnablePassthrough

# 递归式 字符级 文本 切分器
"""
    chunk_size: 建议段落大小
"""
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=200)
# splits = text_splitter.split_documents(docs)

# 底层存储使用的 SQLite，没有指定存储位置的话，则在内存中建立临时库
from langchain_chroma import Chroma
vectorstore = Chroma.from_documents(documents=docs, embedding=embed)

# 把向量操作封装为一个基本检索器
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

from langchain_core.prompts import ChatPromptTemplate

# RAG系统经典的 Prompt (A 增强的过程)
prompt = ChatPromptTemplate.from_messages([
  ("human", """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:""")
])

# 把检索到的4条上下文的文本使用 \n\n 练成一个大的字符串
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# RAG 链
rag_chain = (
    {"context": retriever | format_docs, 
     "question": RunnablePassthrough()}
    | prompt
    | chat
    | StrOutputParser()
)

In [8]:
rag_chain.invoke(input="平均分是多少？")

'无法计算平均分，因为只有Tom和Jerry两个人的成绩，需要全班或更多人的成绩来计算平均分。'

In [9]:
retriever.invoke(input="谁的成绩最高？")

[Document(metadata={'row': 1, 'source': 'students.csv'}, page_content='name: Jerry\nage: 11\nscore: 88'),
 Document(metadata={'row': 0, 'source': 'students.csv'}, page_content='name: Tom\nage: 12\nscore: 77')]

In [19]:
txt_loader = TextLoader(file_path="大语言模型.txt", encoding="utf8")

In [20]:
docs = txt_loader.load()

In [29]:
html_file = "中共中央政治局常务委员会召开会议 研究部署防汛抗洪救灾工作 中共中央总书记习近平主持会议__中国政府网.html"

In [53]:
html_loader = UnstructuredHTMLLoader(file_path=html_file, mode="single")

In [54]:
docs = html_loader.load()

In [60]:
# Load HTML pages using `urllib` and parse them with `BeautifulSoup'.
web_loader = WebBaseLoader(web_path="https://www.gov.cn/yaowen/liebiao/202407/content_6964426.htm")

In [61]:
docs = web_loader.load()

In [63]:
len(docs)

1

In [64]:
docs[0].metadata

{'source': 'https://www.gov.cn/yaowen/liebiao/202407/content_6964426.htm',
 'title': '中共中央政治局常务委员会召开会议\u2002研究部署防汛抗洪救灾工作\u2002中共中央总书记习近平主持会议__中国政府网',
 'description': '中共中央政治局常务委员会7月25日召开会议，研究部署防汛抗洪救灾工作。中共中央总书记习近平主持会议并发表重要讲话。',
 'language': 'No language found.'}

In [66]:
docs[0].page_content

'\n\n\n\n\n\n中共中央政治局常务委员会召开会议\u2002研究部署防汛抗洪救灾工作\u2002中共中央总书记习近平主持会议__中国政府网\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n首页\n|\n简\n|\n繁\n|\nEN\n|\n登录\n\n\n\n个人中心\n退出\n\n\n|\n邮箱\n|\n无障碍\n\n\n\n\n\nEN\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nhttps://www.gov.cn/\n\n\n\n\n\n\n\n\n\n\n\n\n                中共中央政治局常务委员会召开会议\u2002研究部署防汛抗洪救灾工作\u2002中共中央总书记习近平主持会议\n            \n\n                2024-07-25 19:30\n                来源： \n                    新华社\n                \n\n字号：默认\n大\n超大\n\n|\n打印\n|\n\n\n\n\xa0\n\xa0\n\xa0\n\n\n\n\n\n中共中央政治局常务委员会召开会议研究部署防汛抗洪救灾工作中共中央总书记习近平主持会议新华社北京7月25日电\u2002中共中央政治局常务委员会7月25日召开会议，研究部署防汛抗洪救灾工作。中共中央总书记习近平主持会议并发表重要讲话。会议指出，今年我国气候年景偏差，强降雨过程多、历时长，江河洪水发生早、发展快，一些地方反复遭受强降雨冲击，防汛抗洪形势严峻复杂。在以习近平同志为核心的党中央坚强领导下，各级党委和政府迅速行动、全力应对，国家防总、各有关部门和单位履职尽责、通力协作，解放军和武警部队、国家综合性消防救援队伍和各类专业救援力量闻令而动、冲锋在前，广大干部群众风雨同舟、众志成城，共同构筑起了守护家园的坚固防线，防汛抗洪救灾取得重要阶段性成果。会议强调，当前我国正值“七下八上”防汛关键期，长江等流域防洪峰、防决堤、排内涝压力不减，黄河、淮河、海河、松辽流域有可能发生较重汛情，叠加台风进入活跃期，防汛形势更加严峻复杂。各有关地区、部门和单位要始终绷紧防汛抗洪

In [83]:
pdf_loader = PyMuPDFLoader(file_path="YOLOv9.pdf")

In [84]:
docs = pdf_loader.load()

In [85]:
len(docs)

18

In [86]:
docs[0].metadata

{'source': 'YOLOv9.pdf',
 'file_path': 'YOLOv9.pdf',
 'page': 0,
 'total_pages': 18,
 'format': 'PDF 1.5',
 'title': '',
 'author': '',
 'subject': '',
 'keywords': '',
 'creator': 'LaTeX with hyperref',
 'producer': 'pdfTeX-1.40.25',
 'creationDate': 'D:20240222020837Z',
 'modDate': 'D:20240222020837Z',
 'trapped': ''}

In [87]:
print(docs[1].page_content)

Figure 2. Visualization results of random initial weight output feature maps for different network architectures: (a) input image, (b)
PlainNet, (c) ResNet, (d) CSPNet, and (e) proposed GELAN. From the figure, we can see that in different architectures, the information
provided to the objective function to calculate the loss is lost to varying degrees, and our architecture can retain the most complete
information and provide the most reliable gradient information for calculating the objective function.
In deep networks, the phenomenon of input data losing
information during the feedforward process is commonly
known as information bottleneck [59], and its schematic di-
agram is as shown in Figure 2. At present, the main meth-
ods that can alleviate this phenomenon are as follows: (1)
The use of reversible architectures [3, 16, 19]: this method
mainly uses repeated input data and maintains the informa-
tion of the input data in an explicit way; (2) The use of
masked modeling [1, 6, 9, 27

In [88]:
word_loader = UnstructuredWordDocumentLoader(file_path="政府政务和城市治理.docx", mode="single")

In [98]:
docs = word_loader.load()

In [100]:
len(docs)

1

In [102]:
docs[0].metadata

{'source': '政府政务和城市治理.docx'}

In [104]:
print(docs[0].page_content)

政府政务和城市治理

课程大纲

课次 模块 知识点 授课方式 第1次课 1，人工智能技术概论 1，人工智能技术流派； 2，计算机视觉核心技术梳理； 3，自然语言处理核心技术梳理； 4，CV大模型技术； 5，NLP大模型； 理论讲解 2，政府政务案例剖析 1，平安校园： 基于现有视频监控系统，植入AI识别算法，检测并追踪校园内的陌生人、学生危险动作（抽烟、打架、跳楼、持危险品等）、学生情绪（情绪亢奋、正常、抑郁）等，并及时上报，作出预警； 2，智慧农业： 将计算机视觉算法和无人机结合在一起，实现无人机巡航，应用于农产品品质检测、作物识别与分级、农副产品出厂质量监测、植物生长监测、病虫害的探测与防治、自动化收获等领域，为精细农业和农业生产自动化奠定了基础； 3，公众安防系统 动态人脸识别技术则通过摄像头等视觉传感设备在视频流中获得动态的多个人脸图像，从数据库中的大量图像中找到相似度最高的人脸图像，用于人群密集场所当中的布控，协助安全部门进行可疑人口排查、逃犯抓捕等情报研判任务； 4，智慧医疗 从国家层面采集足够的医疗影像数据，采用计算机视觉算法对医疗影像进行自动检测、分类与识别，实现了人眼无法企及的高效与精准，为医生提供了重要的临床决策支持，同时对患者来说，实现了不同级别医院的同质化医疗体验； 5，无人机+AI算法助理自然灾害预警 在雨季，采用无人机巡检，对山川，道路，河流，大堤等高危场景进行自动化监控和预警，捕捉潮汐变化，检测水位、水速、水深；评估泄洪方案，预估受灾范围和重建成本； 案例剖析 3，城市治理案例剖析 1，金山AI鑫眼城市监控平台 依托现有的视频监控，并为前端摄像头加入AI算法，可以动态感知城市里的人、车、事物，解决摄像头“看得见”，但“看不懂”的弊端； 2，早晚高峰交通拥堵预测 基于交通信息建立算法模型，精准预测各关键路段在某个时段的车辆平均通行时间，产出旅行时间预测算法，运用大数据手段缓解拥堵毒瘤对城市管理的影响，更好的帮助贵阳交通管理者提前制定管控方案，预防和削减拥堵，实现智慧出行； AIGC驱动城市治理 软通智慧提出"物联网泛在感知，大数据险合汇聚，大模型理解推理"的城市治理新范式，构建通用人工智能基础设施，通过AI能填补基层人力的不足，泛化感知提高城市治理效率，Plugin连接城市服务，自然语惠理解和生成给市民服务带来温度，使城市治理精准高效，

In [112]:
excel_loader = UnstructuredExcelLoader(file_path="students.xlsx", mode="elements")

In [113]:
docs = excel_loader.load()

'<table border="1" class="dataframe">\n  <tbody>\n    <tr>\n      <td>name</td>\n      <td>age</td>\n      <td>score</td>\n    </tr>\n    <tr>\n      <td>Tom</td>\n      <td>12</td>\n      <td>66</td>\n    </tr>\n    <tr>\n      <td>Jerry</td>\n      <td>13</td>\n      <td>77</td>\n    </tr>\n    <tr>\n      <td>Jim</td>\n      <td>11</td>\n      <td>88</td>\n    </tr>\n  </tbody>\n</table>'

In [119]:
docs[0].metadata["text_as_html"]

'<table border="1" class="dataframe">\n  <tbody>\n    <tr>\n      <td>name</td>\n      <td>age</td>\n      <td>score</td>\n    </tr>\n    <tr>\n      <td>Tom</td>\n      <td>12</td>\n      <td>66</td>\n    </tr>\n    <tr>\n      <td>Jerry</td>\n      <td>13</td>\n      <td>77</td>\n    </tr>\n    <tr>\n      <td>Jim</td>\n      <td>11</td>\n      <td>88</td>\n    </tr>\n  </tbody>\n</table>'

In [120]:
print(docs[0].page_content)




name
age
score


Tom
12
66


Jerry
13
77


Jim
11
88





In [122]:
ppt_loader = UnstructuredPowerPointLoader(file_path="AIGC-智慧引擎队.pptx", mode="single")

In [125]:
docs = ppt_loader.load()

In [128]:
docs[0].metadata

{'source': 'AIGC-智慧引擎队.pptx'}

In [130]:
print(docs[0].page_content)





办公助手

智慧引擎队 （华润电力）

2024-01-12



智慧引擎队团队介绍 

陈旖希

燕志伟

袁桂东

夏彩莲

华润电力

华润电力

华润电力

华润数科

香港中文大学   硕士  

科数部IT平台建设资深经理 

信息系统高级项目管理师 

华润电力第一期优秀青马学员 

华润电力第二期潜质人才班学员

西安交通大学       博士

University of Southern California 

   访问学者

IBM PowerPC 架构应用大赛冠军

河南省科技进步二等奖

广东科学技术职院学院 大专

开发维护资深经理 

2019年度优秀员工

华润电力技术委员会成员

长安大学       学士

华润数科高级咨询经理

PMP 认证项目管理



项目背景

现实的 AIGC 工具箱

理想 AIGC 工具流水线

写材料-累

交材料-急

 AIGC工具-杂

全流程工具-缺

（AIGC 办公助手）



AIGC 办公助手-业务方案

AIGC 办公助手提供 全流程 服务。通过 ModelScope AI 代理框架，在 后台整合 现有 AIGC 工具，一键完成 常见的文案生成。

AI 办公助手

虚拟演讲者

PPT

美工

文档/文案

PPT或方案讲述

生成PPT

文案配图

文档编写

策划

制定文案框架



AIGC工具箱

文案

策划

美工

PPT制作

数字人

ＡIGC  

工具箱



AIGC 办公助手-技术方案

基于大语言模型，通过ModelScope-Agent开源框架创建办公助手，全自动完成办公任务。

React

前端

长期记忆

短期记忆

工具搜索

记忆

反射

代码解释器

自我反思

规划

办公助手

工具

工具调用

思维链

子目标拆解

行动

更多

LLM

底层



案例展示(1/4)-华润电力办公助手平台

将AI应用于《华润电力数字化人才培养方案》案例展示



案例展示（2/4）-任务规划及执行



案例展示（3/4）-图片生成



案例展示（4/4）-PPT生成

将AI应用于《华润电力数字化人才培养方案》撰写及汇报，以下是最终汇报Demo



一体化助手

多Agent一站式服务，全方位支持文案输出


In [131]:
from langchain_community.document_loaders import SQLDatabaseLoader

In [ ]:
SQLDatabaseLoader()

In [132]:
from langchain_community.utilities import SQLDatabase

In [133]:
from sqlalchemy import create_engine

In [134]:
engine = create_engine(url="sqlite:///students.db")

In [135]:
db = SQLDatabase(engine=engine)

In [139]:
sql_loader = SQLDatabaseLoader(query="SELECT * FROM stu_score;", db=db)

In [140]:
docs = sql_loader.load()

In [146]:
print(docs[0].page_content)

id: 1
name: Tom
age: 12
score: 66


### DocumentLoader

- 读取数据
    - 各种格式解析
- 返回结果
  - list
    - Document
      - metadata (dict)
      - page_content (str)

In [154]:
from langchain_core.documents import Document

In [156]:
class MyDocumentLoader(object):
    
    def __init__(self, file_path):
        self.file_path = file_path
    
    def load(self):
        # 任意读取你的数据
        docs = []
        with open(file=self.file_path, mode="r", encoding="utf8") as f:
            for idx, line in enumerate(f):
                doc = Document(metadata=dict(file_name=self.file_path, 
                                            row_num=idx,
                                            author="李晓华"),
                              page_content=line)
                docs.append(doc)
        return docs

In [157]:
my_loader = MyDocumentLoader(file_path="students.csv")

In [158]:
docs = my_loader.load()

In [160]:
docs

[Document(metadata={'file_name': 'students.csv', 'row_num': 0, 'author': '李晓华'}, page_content='name,age,score\n'),
 Document(metadata={'file_name': 'students.csv', 'row_num': 1, 'author': '李晓华'}, page_content='Tom,12,77\n'),
 Document(metadata={'file_name': 'students.csv', 'row_num': 2, 'author': '李晓华'}, page_content='Jerry,11,88\n'),
 Document(metadata={'file_name': 'students.csv', 'row_num': 3, 'author': '李晓华'}, page_content='Jim,12,96')]